In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [13]:
data_dir = '/content/drive/MyDrive/Task7/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)

['ants', 'bees']


In [15]:
def imshow(inp, title):
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [ ]:
inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [16]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model


In [17]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)

model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6777 Acc: 0.5779
val Loss: 0.4639 Acc: 0.8366

Epoch 1/24
----------
train Loss: 0.5492 Acc: 0.7131
val Loss: 0.3678 Acc: 0.8562

Epoch 2/24
----------
train Loss: 0.4656 Acc: 0.7910
val Loss: 0.3066 Acc: 0.8824

Epoch 3/24
----------
train Loss: 0.4398 Acc: 0.8074
val Loss: 0.2704 Acc: 0.8954

Epoch 4/24
----------
train Loss: 0.4178 Acc: 0.7992
val Loss: 0.2441 Acc: 0.9150

Epoch 5/24
----------
train Loss: 0.3893 Acc: 0.8197
val Loss: 0.2282 Acc: 0.9412

Epoch 6/24
----------
train Loss: 0.3468 Acc: 0.8443
val Loss: 0.2223 Acc: 0.9150

Epoch 7/24
----------
train Loss: 0.3810 Acc: 0.8115
val Loss: 0.2050 Acc: 0.9477

Epoch 8/24
----------
train Loss: 0.3497 Acc: 0.8689
val Loss: 0.2150 Acc: 0.9477

Epoch 9/24
----------
train Loss: 0.3507 Acc: 0.8566
val Loss: 0.2258 Acc: 0.9085

Epoch 10/24
----------
train Loss: 0.3402 Acc: 0.8197
val Loss: 0.2130 Acc: 0.9346

Epoch 11/24
----------
train Loss: 0.4006 Acc: 0.8074
val Loss: 0.2246 Acc: 0.9216

Ep

Попробуем теперь оптимизатор Adam

In [20]:
  model = models.resnet18(pretrained=True)
  num_ftrs = model.fc.in_features

  model.fc = nn.Linear(num_ftrs, 2)

  model = model.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

  model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)
  model_conv = torchvision.models.resnet18(pretrained=True)
  for param in model_conv.parameters():
      param.requires_grad = False

  num_ftrs = model_conv.fc.in_features
  model_conv.fc = nn.Linear(num_ftrs, 2)

  model_conv = model_conv.to(device)

  criterion = nn.CrossEntropyLoss()

  optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

  model_conv = train_model(model_conv, criterion, optimizer_conv,
                          exp_lr_scheduler, num_epochs=25)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/24
----------
train Loss: 0.9484 Acc: 0.5533
val Loss: 0.7635 Acc: 0.5425

Epoch 1/24
----------
train Loss: 0.6741 Acc: 0.6311
val Loss: 0.8112 Acc: 0.5948

Epoch 2/24
----------
train Loss: 0.7048 Acc: 0.5902
val Loss: 0.5926 Acc: 0.6732

Epoch 3/24
----------
train Loss: 0.6721 Acc: 0.6066
val Loss: 0.7519 Acc: 0.6013

Epoch 4/24
----------
train Loss: 0.6425 Acc: 0.6598
val Loss: 0.7474 Acc: 0.6471

Epoch 5/24
----------
train Loss: 0.6982 Acc: 0.6148
val Loss: 0.7141 Acc: 0.5425

Epoch 6/24
----------
train Loss: 0.7660 Acc: 0.5410
val Loss: 0.6762 Acc: 0.6275

Epoch 7/24
----------
train Loss: 0.6005 Acc: 0.6926
val Loss: 0.5699 Acc: 0.7124

Epoch 8/24
----------
train Loss: 0.5773 Acc: 0.7254
val Loss: 0.5768 Acc: 0.6993

Epoch 9/24
----------
train Loss: 0.5255 Acc: 0.7787
val Loss: 0.5438 Acc: 0.7516

Epoch 10/24
----------
train Loss: 0.5787 Acc: 0.7131
val Loss: 0.5358 Acc: 0.7451

Epoch 11/24
----------
train Loss: 0.5044 Acc: 0.7582
val Loss: 0.6588 Acc: 0.6797

Ep

А теперь AdamW

In [21]:
  model = models.resnet18(pretrained=True)
  num_ftrs = model.fc.in_features
  
  model.fc = nn.Linear(num_ftrs, 2)

  model = model.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.AdamW(model.parameters(), lr=0.001)

  step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

  model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)
  model_conv = torchvision.models.resnet18(pretrained=True)
  for param in model_conv.parameters():
      param.requires_grad = False

  num_ftrs = model_conv.fc.in_features
  model_conv.fc = nn.Linear(num_ftrs, 2)

  model_conv = model_conv.to(device)

  criterion = nn.CrossEntropyLoss()

  optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

  model_conv = train_model(model_conv, criterion, optimizer_conv,
                          exp_lr_scheduler, num_epochs=25)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/24
----------
train Loss: 1.0824 Acc: 0.5943
val Loss: 0.8644 Acc: 0.5556

Epoch 1/24
----------
train Loss: 0.7421 Acc: 0.5656
val Loss: 4.0947 Acc: 0.4575

Epoch 2/24
----------
train Loss: 0.7768 Acc: 0.5656
val Loss: 0.9671 Acc: 0.5621

Epoch 3/24
----------
train Loss: 0.7593 Acc: 0.5246
val Loss: 0.6858 Acc: 0.6732

Epoch 4/24
----------
train Loss: 0.7303 Acc: 0.5779
val Loss: 0.6640 Acc: 0.5686

Epoch 5/24
----------
train Loss: 0.6944 Acc: 0.5697
val Loss: 0.7608 Acc: 0.6732

Epoch 6/24
----------
train Loss: 0.7257 Acc: 0.6066
val Loss: 0.7557 Acc: 0.5686

Epoch 7/24
----------
train Loss: 0.6973 Acc: 0.5492
val Loss: 0.7175 Acc: 0.6144

Epoch 8/24
----------
train Loss: 0.6554 Acc: 0.6557
val Loss: 0.6684 Acc: 0.6144

Epoch 9/24
----------
train Loss: 0.6305 Acc: 0.6475
val Loss: 0.7403 Acc: 0.6013

Epoch 10/24
----------
train Loss: 0.6415 Acc: 0.6189
val Loss: 0.7005 Acc: 0.6209

Epoch 11/24
----------
train Loss: 0.6273 Acc: 0.6475
val Loss: 0.6833 Acc: 0.6405

Ep

Лучший результат показал TransferLearning с оптимизатором Adam